# 4.1 트랜스퍼러닝(이어서)
- 작성자: [김찬란](https://github.com/seriousran)

## 4.1.4 구분하는 능력을 더 강력하게 학습하기 - 메트릭 러닝

In [0]:
# 코드 4-4 트리플렛 손실 함수

def triplet_loss(y_true, y_pred):
  margin = K.constant(1)
  return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0]) - 0.5 * (K.square(y_pred[:,1,0]) + K.square(y_pred[:,2,0])) + margin))

## 4.1.5 트리플렛 데이터셋분할하기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 코드 4-5 폴더 이름 가져오기 및 정렬

import os
import glob
from shutil import copyfile
from random import shuffle

folder_list = glob.glob("/content/drive/My Drive/colab/oxford-flowers-17/*/")
folder_list.sort()

In [0]:
# 코드 4-6 이미지 파일 이름 가져오기

whole_img_list = glob.glob("/content/drive/My Drive/colab/oxford-flowers-17/*/*.jpg")

In [16]:
# 코드 4-7 앵커 데이터 생성

BASE_PATH = "/content/drive/My Drive/colab/"

if not os.path.exists(BASE_PATH + 'flowers_anchor'):
  os.mkdir(BASE_PATH + 'flowers_anchor')
if not os.path.exists(BASE_PATH + 'flowers_anchor/images'):
  os.mkdir(BASE_PATH + 'flowers_anchor/images')

flower_list = []
for folder in folder_list:
  img_list = glob.glob(folder + "*.jpg")
  tmp_list = img_list.copy()
  flower_list.append(img_list)

for i, flower in enumerate(flower_list):
  for j, img in enumerate(flower):
    copyfile(img, '/'.join(img.split('/')[:5]) + '/flowers_anchor/images/image_' + str(i*len(flower) + j).zfill(4) + '.jpg')

positive_list = glob.glob("/content/drive/My Drive/colab/flowers_anchor/images/*.jpg")
len(positive_list)

1360

In [17]:
# 코드 4-8 긍정 데이터 생성

def shuffle_not_in_place(input_list):
    randomized_list = input_list[:]
    while True:
        shuffle(randomized_list)
        for a, b in zip(input_list, randomized_list):
            if a == b:
                break
        else:
            return randomized_list

if not os.path.exists(BASE_PATH + 'flowers_positive'):
  os.mkdir(BASE_PATH + 'flowers_positive')
if not os.path.exists(BASE_PATH + 'flowers_positive/images'):
  os.mkdir(BASE_PATH + 'flowers_positive/images')

flower_list = []
for folder in folder_list:
  img_list = glob.glob(folder + "*.jpg")
  img_list = shuffle_not_in_place(img_list)
  flower_list.append(img_list)

for i, flower in enumerate(flower_list):
  for j, img in enumerate(flower):
    copyfile(img, '/'.join(img.split('/')[:5]) + '/flowers_positive/images/image_' + str(i*len(flower) + j).zfill(4) + '.jpg')

positive_list = glob.glob("/content/drive/My Drive/colab/flowers_positive/images/*.jpg")
len(positive_list)

1360

In [18]:
# 코드 4-9 부정 데이터 생성

folder_names = [folder.split('/')[6] for folder in folder_list]

if not os.path.exists(BASE_PATH + 'flowers_negative'):
  os.mkdir(BASE_PATH + 'flowers_negative')
if not os.path.exists(BASE_PATH + 'flowers_negative/images'):
  os.mkdir(BASE_PATH + 'flowers_negative/images')

whole_shuffled = whole_img_list.copy()
shuffle(whole_shuffled)
i=0
j=0
while j < len(folder_names):
  if folder_names[j] == whole_shuffled[i].split('/')[6]:
    tmp = whole_shuffled.pop(i)
    whole_shuffled.append(tmp)
  else:
    i+=1
    if i%80 == 0:
      j+=1

for i, img in enumerate(whole_shuffled):
  copyfile(img, '/'.join(img.split('/')[:5]) + '/flowers_negative/images/image_' + str(i).zfill(4) + '.jpg')

negative = glob.glob("/content/drive/My Drive/colab/flowers_negative/images/*.jpg")
len(negative)

1360